# Задание по продуктовой аналитике

In [45]:
import pandas as pd

pd.set_option('display.max_columns', None)
deals_df = pd.read_pickle("deals_cleaned.pkl")
deals_df

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Campaign,SLA,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,German level,Create_to_Close_Days,Latitude,Longitude,SLA_range
0,5805028000056864695,Ben Hall,NaT,NaN,New Lead,NaN,03.07.23women,NaT,facebook ads,NaN,NaN,NaN,2024-06-21 15:30:00,<NA>,<NA>,NaN,NaN,5805028000056849495,NaN,NaN,NaN,NaN,NaN,NaN
1,5805028000056859489,Ulysses Adams,NaT,NaN,New Lead,NaN,NaN,NaT,organic,NaN,Web Developer,Morning,2024-06-21 15:23:00,6,<NA>,NaN,"2,000.00",5805028000056834471,NaN,NaN,NaN,NaN,NaN,NaN
2,5805028000056832357,Ulysses Adams,2024-06-21,D - Non Target,Lost,Non target,engwien_at,0 days 00:26:43,telegram posts,NaN,NaN,NaN,2024-06-21 14:45:00,<NA>,<NA>,NaN,NaN,5805028000056854421,NaN,NaN,0.00,NaN,NaN,Medium (15-30 min)
3,5805028000056824246,Eva Kent,2024-06-21,E - Non Qualified,Lost,Invalid number,04.07.23recentlymoved_de,0 days 01:00:04,facebook ads,NaN,NaN,NaN,2024-06-21 13:32:00,<NA>,<NA>,NaN,NaN,5805028000056889351,NaN,NaN,0.00,NaN,NaN,Very slow (>60 min)
4,5805028000056873292,Ben Hall,2024-06-21,D - Non Target,Lost,Non target,discovery_de,0 days 00:53:12,google ads,NaN,NaN,NaN,2024-06-21 13:21:00,<NA>,<NA>,NaN,NaN,5805028000056876176,NaN,NaN,0.00,NaN,NaN,Slow (30-60 min)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21406,5805028000050001603,Quincy Vincent,2024-05-15,E - Non Qualified,Lost,Duplicate,NaN,0 days 05:35:41,organic,NaN,NaN,NaN,2024-05-15 08:50:00,<NA>,<NA>,NaN,NaN,5805028000049855125,NaN,NaN,0.00,NaN,NaN,Very slow (>60 min)
21407,5805028000053787195,Paula Underwood,2024-06-06,E - Non Qualified,Lost,Duplicate,NaN,NaT,organic,NaN,NaN,NaN,2024-06-06 18:45:00,<NA>,<NA>,NaN,NaN,5805028000053705642,NaN,NaN,0.00,NaN,NaN,NaN
21408,5805028000054823001,Quincy Vincent,2024-06-12,E - Non Qualified,Lost,Duplicate,brand_search_eng_de,0 days 04:30:17,google ads,NaN,NaN,NaN,2024-06-12 05:46:00,<NA>,<NA>,NaN,NaN,5805028000054480760,NaN,NaN,0.00,NaN,NaN,Very slow (>60 min)
21409,5805028000056685164,Charlie Davis,2024-06-20,E - Non Qualified,Lost,Duplicate,NaN,0 days 00:01:46,organic,NaN,NaN,NaN,2024-06-20 13:49:00,<NA>,<NA>,NaN,NaN,5805028000056392363,NaN,NaN,0.00,NaN,NaN,Very fast (≤5 min)


In [46]:
deals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21411 entries, 0 to 21410
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype          
---  ------                --------------  -----          
 0   Id                    21411 non-null  object         
 1   Deal Owner Name       21382 non-null  object         
 2   Closing Date          16685 non-null  datetime64[ns] 
 3   Quality               19158 non-null  category       
 4   Stage                 21411 non-null  category       
 5   Lost Reason           15942 non-null  category       
 6   Campaign              16006 non-null  object         
 7   SLA                   15504 non-null  timedelta64[ns]
 8   Source                21411 non-null  object         
 9   Payment Type          496 non-null    category       
 10  Product               3592 non-null   category       
 11  Education Type        3299 non-null   category       
 12  Created Time          21411 non-null  datetime64[ns] 
 13  C

$$
AOV_i = \frac{\text{Initial Amount Paid} + \left(\text{Offer Total Amount} - \text{Initial Amount Paid}\right)}{\text{Course duration} - 1} \cdot (\text{Months of study} - 1) \div \text{Months of study}
$$

In [47]:
deals_df['Course duration'] = pd.to_numeric(deals_df['Course duration'], errors='coerce')
deals_df['Course duration'] = deals_df['Course duration'].fillna(0).astype(int)

In [48]:
import numpy as np

# Условия для валидных строк
valid = (
    deals_df['Initial Amount Paid'].notna() &
    deals_df['Offer Total Amount'].notna() &
    deals_df['Course duration'].notna() &
    deals_df['Months of study'].notna() &
    (deals_df['Course duration'] != 1) &
    (deals_df['Months of study'] != 0)
)

# Расчёт AOV_i с приведением к float64
deals_df['AOV_i'] = np.where(
    valid,
    (
        deals_df['Initial Amount Paid'] +
        ((deals_df['Offer Total Amount'] - deals_df['Initial Amount Paid']) /
         (deals_df['Course duration'] - 1)) *
        (deals_df['Months of study'] - 1)
    ) / deals_df['Months of study'],
    np.nan
).astype('float64') #Т.к. тип данных получается Float64 (с большой буквы), то это может испортить данные (возвращает pd.NA вместо pd.nan и ломает вычисления даже при отсутствии строк с NaN), поэтому меняем тип на float64:

In [49]:
deals_df[deals_df['AOV_i'].notna()]

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Campaign,SLA,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,German level,Create_to_Close_Days,Latitude,Longitude,SLA_range,AOV_i
463,5805028000055502703,Eva Kent,2024-06-18,B - Medium,Payment Done,NaN,performancemax_wide_at,0 days 14:35:20,google ads,Recurring Payments,Digital Marketing,Evening,2024-06-15 20:01:00,11,1,500.00,"4,500.00",5805028000055471387,Podskalie,NaN,3.00,48.17,18.32,Very slow (>60 min),500.00
473,5805028000055524315,Ben Hall,2024-06-19,A - High,Payment Done,NaN,12.07.2023wide_de,0 days 00:17:25,tiktok ads,Recurring Payments,Digital Marketing,Evening,2024-06-15 17:02:00,11,1,500.00,"4,500.00",5805028000055503302,Zinnowitz,NaN,4.00,54.08,13.91,Medium (15-30 min),500.00
491,5805028000055524040,Ben Hall,2024-06-23,B - Medium,Payment Done,NaN,live_de,0 days 00:23:42,telegram posts,NaN,Web Developer,Morning,2024-06-15 12:20:00,6,1,"1,000.00","9,000.00",5805028000055464156,Wolfsburg,B1,8.00,52.42,10.79,Medium (15-30 min),"1,000.00"
824,5805028000054664018,Ben Hall,2024-06-19,D - Non Target,Payment Done,NaN,NaN,0 days 16:31:52,organic,NaN,Web Developer,Morning,2024-06-11 19:39:00,6,1,"1,000.00","9,000.00",5805028000054683001,Zwickau,A2,8.00,50.72,12.49,Very slow (>60 min),"1,000.00"
1029,5805028000054257317,Quincy Vincent,2024-06-17,B - Medium,Payment Done,NaN,02.07.23wide_de,0 days 00:45:55,facebook ads,NaN,Digital Marketing,Morning,2024-06-10 15:33:00,11,1,"1,000.00","11,000.00",5805028000054284461,Leinfelden-Echterdingen,B1,7.00,48.69,9.15,Slow (30-60 min),"1,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21034,5805028000002302112,Julia Nelson,2023-07-19,C - Low,Payment Done,Expensive,performancemax_digitalmarkt_ru_de,0 days 15:24:52,google ads,NaN,Web Developer,Morning,2023-07-18 21:11:00,6,2,"1,000.00","5,000.00",5805028000002302109,Wolfsburg,B1,1.00,52.42,10.79,Very slow (>60 min),900.00
21087,5805028000001987082,Julia Nelson,2023-07-17,C - Low,Payment Done,Conditions are not suitable,12.07.2023wide_de,0 days 00:49:16,tiktok ads,NaN,Digital Marketing,Morning,2023-07-17 18:02:00,11,11,"1,000.00","11,000.00",5805028000001986077,NaN,B1,0.00,NaN,NaN,Slow (30-60 min),"1,000.00"
21130,5805028000001885076,Jane Smith,2023-08-31,A - High,Payment Done,NaN,04.07.23recentlymoved_de,0 days 00:13:31,facebook ads,One Payment,Digital Marketing,Morning,2023-07-15 13:27:00,11,11,450.00,"4,000.00",5805028000001880249,Ingolstadt,NaN,47.00,48.77,11.42,Fast (5-15 min),363.64
21265,5805028000001401001,Oliver Taylor,2023-07-16,B - Medium,Payment Done,NaN,02.07.23wide_de,0 days 02:22:36,facebook ads,NaN,Digital Marketing,NaN,2023-07-08 08:56:00,11,8,"1,000.00","11,500.00",5805028000001350049,NaN,NaN,8.00,NaN,NaN,Very slow (>60 min),"1,043.75"


$$
Revenue_i = AOV_i \times \text{Months of study}
$$

In [50]:
deals_df['Revenue_i'] = (deals_df['AOV_i'] * deals_df['Months of study']).astype('float64') #Т.к. тип данных получается Float64 (с большой буквы), то это может испортить данные (возвращает pd.NA вместо pd.nan и ломает вычисления даже при отсутствии строк с NaN), поэтому меняем тип на float64:

In [51]:
deals_df[deals_df['AOV_i'].notna()]

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Campaign,SLA,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,German level,Create_to_Close_Days,Latitude,Longitude,SLA_range,AOV_i,Revenue_i
463,5805028000055502703,Eva Kent,2024-06-18,B - Medium,Payment Done,NaN,performancemax_wide_at,0 days 14:35:20,google ads,Recurring Payments,Digital Marketing,Evening,2024-06-15 20:01:00,11,1,500.00,"4,500.00",5805028000055471387,Podskalie,NaN,3.00,48.17,18.32,Very slow (>60 min),500.00,500.00
473,5805028000055524315,Ben Hall,2024-06-19,A - High,Payment Done,NaN,12.07.2023wide_de,0 days 00:17:25,tiktok ads,Recurring Payments,Digital Marketing,Evening,2024-06-15 17:02:00,11,1,500.00,"4,500.00",5805028000055503302,Zinnowitz,NaN,4.00,54.08,13.91,Medium (15-30 min),500.00,500.00
491,5805028000055524040,Ben Hall,2024-06-23,B - Medium,Payment Done,NaN,live_de,0 days 00:23:42,telegram posts,NaN,Web Developer,Morning,2024-06-15 12:20:00,6,1,"1,000.00","9,000.00",5805028000055464156,Wolfsburg,B1,8.00,52.42,10.79,Medium (15-30 min),"1,000.00","1,000.00"
824,5805028000054664018,Ben Hall,2024-06-19,D - Non Target,Payment Done,NaN,NaN,0 days 16:31:52,organic,NaN,Web Developer,Morning,2024-06-11 19:39:00,6,1,"1,000.00","9,000.00",5805028000054683001,Zwickau,A2,8.00,50.72,12.49,Very slow (>60 min),"1,000.00","1,000.00"
1029,5805028000054257317,Quincy Vincent,2024-06-17,B - Medium,Payment Done,NaN,02.07.23wide_de,0 days 00:45:55,facebook ads,NaN,Digital Marketing,Morning,2024-06-10 15:33:00,11,1,"1,000.00","11,000.00",5805028000054284461,Leinfelden-Echterdingen,B1,7.00,48.69,9.15,Slow (30-60 min),"1,000.00","1,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21034,5805028000002302112,Julia Nelson,2023-07-19,C - Low,Payment Done,Expensive,performancemax_digitalmarkt_ru_de,0 days 15:24:52,google ads,NaN,Web Developer,Morning,2023-07-18 21:11:00,6,2,"1,000.00","5,000.00",5805028000002302109,Wolfsburg,B1,1.00,52.42,10.79,Very slow (>60 min),900.00,"1,800.00"
21087,5805028000001987082,Julia Nelson,2023-07-17,C - Low,Payment Done,Conditions are not suitable,12.07.2023wide_de,0 days 00:49:16,tiktok ads,NaN,Digital Marketing,Morning,2023-07-17 18:02:00,11,11,"1,000.00","11,000.00",5805028000001986077,NaN,B1,0.00,NaN,NaN,Slow (30-60 min),"1,000.00","11,000.00"
21130,5805028000001885076,Jane Smith,2023-08-31,A - High,Payment Done,NaN,04.07.23recentlymoved_de,0 days 00:13:31,facebook ads,One Payment,Digital Marketing,Morning,2023-07-15 13:27:00,11,11,450.00,"4,000.00",5805028000001880249,Ingolstadt,NaN,47.00,48.77,11.42,Fast (5-15 min),363.64,"4,000.00"
21265,5805028000001401001,Oliver Taylor,2023-07-16,B - Medium,Payment Done,NaN,02.07.23wide_de,0 days 02:22:36,facebook ads,NaN,Digital Marketing,NaN,2023-07-08 08:56:00,11,8,"1,000.00","11,500.00",5805028000001350049,NaN,NaN,8.00,NaN,NaN,Very slow (>60 min),"1,043.75","8,350.00"


In [52]:
print(deals_df['Revenue_i'].dtype)
print(deals_df['AOV_i'].dtype)

float64
float64


$$
AOV = \frac{\sum Revenue_i}{\sum \text{Months of study}}
$$

In [53]:
aov = (
    deals_df['Revenue_i'].sum(skipna=True) /
    deals_df['Months of study'].sum(skipna=True)
)
print(f"AOV = {aov:.2f}")

AOV = 783.21


In [54]:
contacts_df = pd.read_pickle("contacts_cleaned.pkl")
contacts_df

,Id,Contact Owner Name,Created Time,Modified Time
0,5805028000000645014,Rachel White,2023-06-27 11:28:00,2023-12-22 13:34:00
1,5805028000000872003,Charlie Davis,2023-07-03 11:31:00,2024-05-21 10:23:00
2,5805028000000889001,Bob Brown,2023-07-02 22:37:00,2023-12-21 13:17:00
3,5805028000000907006,Bob Brown,2023-07-03 05:44:00,2023-12-29 15:20:00
4,5805028000000939010,Nina Scott,2023-07-04 10:11:00,2024-04-16 16:14:00
...,...,...,...,...
18543,5805028000056889209,Ulysses Adams,2024-06-21 12:11:00,2024-06-21 14:11:00
18544,5805028000056889351,Eva Kent,2024-06-21 13:32:00,2024-06-21 15:32:00
18545,5805028000056892018,Eva Kent,2024-06-21 10:21:00,2024-06-21 12:21:00
18546,5805028000056892055,Yara Edwards,2024-06-21 10:22:00,2024-06-21 12:23:00


In [55]:
spend_df = pd.read_pickle("spend_cleaned.pkl")
spend_df

,Date,Source,Campaign,Impressions,Spend,Clicks,AdGroup
0,2023-07-03,google ads,gen_analyst_de,6,0.00,0,NaN
1,2023-07-03,google ads,performancemax_eng_de,4,0.01,1,NaN
2,2023-07-03,facebook ads,03.07.23women,187,3.30,6,women
3,2023-07-03,facebook ads,03.07.23women,4,0.02,1,women
4,2023-07-03,facebook ads,02.07.23wide_de,61,0.58,0,wide
...,...,...,...,...,...,...,...
16395,2024-06-21,tiktok ads,22.05.2024wide_de,7,0.03,0,wide
16396,2024-06-21,youtube ads,youtube_shorts_de,90,0.51,2,com_august
16397,2024-06-21,facebook ads,17.03.24wide_at,7,0.07,0,wide
16398,2024-06-21,tiktok ads,12.07.2023wide_de,61,0.16,0,wide


In [56]:
deals_df['Contact Name'].nunique()

18089

In [72]:
deals_df[deals_df['Months of study'].isna()]

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Campaign,SLA,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,German level,Create_to_Close_Days,Latitude,Longitude,SLA_range,AOV_i,Revenue_i
0,5805028000056864695,Ben Hall,NaT,NaN,New Lead,NaN,03.07.23women,NaT,facebook ads,NaN,NaN,NaN,2024-06-21 15:30:00,0,<NA>,NaN,NaN,5805028000056849495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5805028000056859489,Ulysses Adams,NaT,NaN,New Lead,NaN,NaN,NaT,organic,NaN,Web Developer,Morning,2024-06-21 15:23:00,6,<NA>,NaN,"2,000.00",5805028000056834471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5805028000056832357,Ulysses Adams,2024-06-21,D - Non Target,Lost,Non target,engwien_at,0 days 00:26:43,telegram posts,NaN,NaN,NaN,2024-06-21 14:45:00,0,<NA>,NaN,NaN,5805028000056854421,NaN,NaN,0.00,NaN,NaN,Medium (15-30 min),NaN,NaN
3,5805028000056824246,Eva Kent,2024-06-21,E - Non Qualified,Lost,Invalid number,04.07.23recentlymoved_de,0 days 01:00:04,facebook ads,NaN,NaN,NaN,2024-06-21 13:32:00,0,<NA>,NaN,NaN,5805028000056889351,NaN,NaN,0.00,NaN,NaN,Very slow (>60 min),NaN,NaN
4,5805028000056873292,Ben Hall,2024-06-21,D - Non Target,Lost,Non target,discovery_de,0 days 00:53:12,google ads,NaN,NaN,NaN,2024-06-21 13:21:00,0,<NA>,NaN,NaN,5805028000056876176,NaN,NaN,0.00,NaN,NaN,Slow (30-60 min),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21406,5805028000050001603,Quincy Vincent,2024-05-15,E - Non Qualified,Lost,Duplicate,NaN,0 days 05:35:41,organic,NaN,NaN,NaN,2024-05-15 08:50:00,0,<NA>,NaN,NaN,5805028000049855125,NaN,NaN,0.00,NaN,NaN,Very slow (>60 min),NaN,NaN
21407,5805028000053787195,Paula Underwood,2024-06-06,E - Non Qualified,Lost,Duplicate,NaN,NaT,organic,NaN,NaN,NaN,2024-06-06 18:45:00,0,<NA>,NaN,NaN,5805028000053705642,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN
21408,5805028000054823001,Quincy Vincent,2024-06-12,E - Non Qualified,Lost,Duplicate,brand_search_eng_de,0 days 04:30:17,google ads,NaN,NaN,NaN,2024-06-12 05:46:00,0,<NA>,NaN,NaN,5805028000054480760,NaN,NaN,0.00,NaN,NaN,Very slow (>60 min),NaN,NaN
21409,5805028000056685164,Charlie Davis,2024-06-20,E - Non Qualified,Lost,Duplicate,NaN,0 days 00:01:46,organic,NaN,NaN,NaN,2024-06-20 13:49:00,0,<NA>,NaN,NaN,5805028000056392363,NaN,NaN,0.00,NaN,NaN,Very fast (≤5 min),NaN,NaN


## Расчет общей юнит-экономики

In [64]:
# -- поток юнитов масштабирования --
ua = contacts_df['Id'].nunique() 

# -- маректинговый бюджет -- 
ac = spend_df['Spend'].sum()

# -- стоимость привлечения юнита масштабирования --
ltc_cpa = ac / ua

# -- количество клиентов -- 
b = deals_df[
    (deals_df['Stage'] == 'Payment Done') &
    (deals_df['Initial Amount Paid'].notna()) &
    (deals_df['Months of study'].notna())
]['Contact Name'].nunique()

# -- конверсия из лида в клиента -- 
c1 = b / ua

# -- количество транзакций --
t = deals_df[
    (deals_df['Stage'] == 'Payment Done') &
    (deals_df['Initial Amount Paid'].notna()) 
]['Months of study'].sum()

# -- среднее число сделок на одного клиента --
apc = t / b

# -- средняя валовая прибыль на клиента --
cltv = aov * apc

# -- средняя валовая прибыль на юнит масштабирования --
ltv = cltv * c1

# Стоимость привлечения клиента
cac = ac / b

# -- выручка --
revenue = b * aov * apc #deals_df['Revenue_i'].sum(skipna=True)

# -- маржинальная прибыль в юнит-экономике --
cm = ua * (ltv - ltc_cpa)

print(f'''AOV: {aov:.2f}
UA: {ua}
LTC (CPA): {ltc_cpa:.2f}
AC: {ac}
C1: {c1:.4f}
B: {b}
T: {t}
APC: {apc:.2f}
CLTV: {cltv:.2f}
LTV: {ltv:.2f}
CAC: {cac:.2f}
Revenue: {revenue:.2f}
CM: {cm:.2f}
''')

AOV: 783.21
UA: 18548
LTC (CPA): 8.06
AC: 149523.45
C1: 0.0446
B: 827
T: 4561
APC: 5.52
CLTV: 4319.47
LTV: 192.59
CAC: 180.80
Revenue: 3572199.74
CM: 3422676.29



In [60]:
# Соберём всё в датафрейм
metrics = {
    'Metric': [
        'AOV', 'UA', 'LTC (CPA)', 'AC', 'C1', 'B', 'T', 'APC',
        'CLTV', 'LTV', 'CAC', 'Revenue', 'CM'
    ],
    'Value': [
        round(aov, 2), ua, round(ltc_cpa, 2), round(ac, 2), round(c1, 4), b, t,
        round(apc, 2), round(cltv, 2), round(ltv, 2), round(cac, 2),
        round(revenue, 2), round(cm, 2)
    ]
}
metrics_df = pd.DataFrame(metrics)

metrics_df['Value'] = metrics_df['Value'].apply(
    lambda x: f"{x:,.2f}" if isinstance(x, float) else f"{x:,}"
)

In [61]:
metrics_df

,Metric,Value
0,AOV,783.21
1,UA,"18,548.00"
2,LTC (CPA),8.06
3,AC,"149,523.45"
4,C1,0.04
5,B,827.00
6,T,"4,561.00"
7,APC,5.52
8,CLTV,"4,319.47"
9,LTV,192.59


In [62]:
metrics_df.to_pickle('metrics.pkl')

In [65]:
import pandas as pd

def calculate_unit_economics_by_product(deals_df, contacts_df, spend_df):
    # Общие метрики
    total_ua = contacts_df['Id'].nunique()
    total_ac = spend_df['Spend'].sum()
    total_ltc_cpa = total_ac / total_ua if total_ua != 0 else 0

    results = []

    # Только успешные сделки
    success_deals = deals_df[
        (deals_df['Stage'] == 'Payment Done') &
        (deals_df['Initial Amount Paid'].notna())&
        (deals_df['Months of study'].notna())
    ]

    # Группировка только по Product, исключаем пустые значения
    grouped = success_deals.dropna(subset=['Product']).groupby('Product', observed=True)

    for product, group in grouped:
        buyers = group['Contact Name'].nunique()
        c1 = buyers / total_ua if total_ua != 0 else 0
        total_months = group['Months of study'].sum()
        apc = total_months / buyers if buyers != 0 else 0
        aov = group['Revenue_i'].sum() / total_months if total_months != 0 else 0
        cltv = aov * apc
        ltv = cltv * c1
        cac = total_ac / buyers if buyers != 0 else 0
        revenue = buyers * aov * apc
        cm = total_ua * (ltv - total_ltc_cpa)

        results.append({
            'Product': product,
            'AOV': aov,
            'UA': total_ua,
            'LTC (CPA)': total_ltc_cpa,
            'AC': total_ac,
            'C1': c1,
            'B': buyers,
            'T': total_months,
            'APC': apc,
            'CLTV': cltv,
            'LTV': ltv,
            'CAC': cac,
            'Revenue': revenue,
            'CM': cm
        })

    return pd.DataFrame(results)

product_df = calculate_unit_economics_by_product(deals_df, contacts_df, spend_df)
product_df

,Product,AOV,UA,LTC (CPA),AC,C1,B,T,APC,CLTV,LTV,CAC,Revenue,CM
0,Digital Marketing,783.95,18548,8.06,"149,523.45",0.03,468,2886,6.17,"4,834.38",121.98,319.49,"2,262,490.00","2,112,966.55"
1,UX/UI Design,813.37,18548,8.06,"149,523.45",0.01,226,1170,5.18,"4,210.82",51.31,661.61,"951,645.00","802,121.55"
2,Web Developer,726.10,18548,8.06,"149,523.45",0.01,135,505,3.74,"2,716.15",19.77,"1,107.58","366,680.00","217,156.55"


In [27]:
pd.set_option('display.float_format', '{:,.3f}'.format)

In [36]:
import pandas as pd

def calculate_unit_economics_by_group(deals_df, contacts_df, spend_df):
    # Общие метрики
    ua_prod = contacts_df['Id'].nunique()
    ac_prod = spend_df['Spend'].sum()
    ltc_cpa_prod = ac_prod / ua_prod if ua_prod != 0 else 0

    results = []

    success_deals = deals_df[
    (deals_df['Stage'] == 'Payment Done') &
    (deals_df['Initial Amount Paid'].notna())
]

    # Группировка по Product и Education Type, без NaN
    grouped = success_deals.dropna(subset=['Product', 'Education Type']) \
                      .groupby(['Product', 'Education Type'], observed=True)

    for (product, edu_type), df_group in grouped:
        b_prod = df_group[
            (df_group['Stage'] == 'Payment Done') &
            (df_group['Initial Amount Paid'].notna())
        ]['Contact Name'].nunique()

        c1_prod = b_prod / ua_prod if ua_prod != 0 else 0
        t_prod = df_group['Months of study'].sum()
        apc_prod = t_prod / b_prod if b_prod != 0 else 0
        aov_prod = df_group['Revenue_i'].sum() / t_prod if t_prod != 0 else 0
        cltv_prod = aov_prod * apc_prod
        ltv_prod = cltv_prod * c1_prod
        cac_prod = ac_prod / b_prod
        revenue_prod = b_prod * aov_prod * apc_prod
        cm_prod = ua_prod * (ltv_prod - ltc_cpa_prod)
       
        results.append({
            'Product': product,
            'Education Type': edu_type,
            'AOV': aov_prod,
            'UA': ua_prod,
            'LTC (CPA)': ltc_cpa_prod,
            'AC': ac_prod,
            'C1': c1_prod,
            'B': b_prod,
            'T': t_prod,
            'APC': apc_prod,
            'CLTV': cltv_prod,
            'LTV': ltv_prod,   
            'CAC': cac_prod,
            'Revenue': revenue_prod,
            'CM': cm_prod
        })

    unit_economics_by_group_df = pd.DataFrame(results)
    return unit_economics_by_group_df

unit_df = calculate_unit_economics_by_group(deals_df, contacts_df, spend_df)
pd.set_option('display.float_format', '{:,.2f}'.format)
unit_df

,Product,Education Type,AOV,UA,LTC (CPA),AC,C1,B,T,APC,CLTV,LTV,CAC,Revenue,CM
0,Digital Marketing,Evening,365.69,18548,8.06,"149,523.45",0.01,112,764,6.82,"2,494.55",15.06,"1,335.03","279,390.00","129,866.55"
1,Digital Marketing,Morning,931.25,18548,8.06,"149,523.45",0.02,349,2056,5.89,"5,486.10",103.23,428.43,"1,914,650.00","1,765,126.55"
2,UX/UI Design,Evening,432.87,18548,8.06,"149,523.45",0.00,58,284,4.90,"2,119.57",6.63,"2,577.99","122,935.00","-26,588.45"
3,UX/UI Design,Morning,935.34,18548,8.06,"149,523.45",0.01,169,886,5.24,"4,903.61",44.68,884.75,"828,710.00","679,186.55"
4,Web Developer,Morning,726.10,18548,8.06,"149,523.45",0.01,135,505,3.74,"2,716.15",19.77,"1,107.58","366,680.00","217,156.55"


In [66]:
def simulate_metric_changes_by_product(df, product_name, delta_pct=10):
    increase_factor = 1 + delta_pct / 100
    decrease_factor = 1 - delta_pct / 100

    # Классификация метрик
    metrics = ['AOV', 'UA', 'AC', 'LTC (CPA)', 'C1', 'B', 'T', 'APC', 'CLTV', 'LTV', 'Revenue', 'CM']
    metrics_to_increase = ['AOV', 'UA', 'C1', 'APC']
    metrics_to_decrease = ['LTC (CPA)']
    metrics_fixed = ['AC', 'B', 'T', 'CLTV', 'LTV']  # рассчитываются, но не меняются напрямую
    row_metrics = metrics_to_increase + metrics_to_decrease

    # Получение базовых значений
    base_row = df[df['Product'] == product_name].iloc[0]
    base_values = base_row[metrics].to_dict()

    # Подготовка таблицы
    table = {
        'Metric ↑ / ↓ →': ['Base'] + [
            f'{m} +{delta_pct}%' if m in metrics_to_increase else f'{m} -{delta_pct}%' for m in row_metrics
        ]
    }

    for column_metric in metrics:
        values = [base_values[column_metric]]

        for changed_metric in row_metrics:
            temp = base_values.copy()

            # Применение изменений
            if changed_metric in metrics_to_increase:
                temp[changed_metric] *= increase_factor
            elif changed_metric in metrics_to_decrease:
                temp[changed_metric] *= decrease_factor

            # Пересчёт зависимых метрик
            temp['CLTV'] = temp['AOV'] * temp['APC']
            temp['LTV'] = temp['CLTV'] * temp['C1']
            temp['CM'] = temp['UA'] * (temp['LTV'] - temp['LTC (CPA)'])
            temp['Revenue'] = temp['AOV'] * temp['APC'] * temp['B']

            values.append(temp[column_metric])

        table[column_metric] = values

    return pd.DataFrame(table)

In [67]:
simulate_metric_changes_by_product(product_df, 'Digital Marketing', delta_pct=10)

,Metric ↑ / ↓ →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,783.95,"18,548.00","149,523.45",8.06,0.03,468,2886,6.17,"4,834.38",121.98,"2,262,490.00","2,112,966.55"
1,AOV +10%,862.35,"18,548.00","149,523.45",8.06,0.03,468,2886,6.17,"5,317.82",134.18,"2,488,739.00","2,339,215.55"
2,UA +10%,783.95,"20,402.80","149,523.45",8.06,0.03,468,2886,6.17,"4,834.38",121.98,"2,262,490.00","2,324,263.21"
3,C1 +10%,783.95,"18,548.00","149,523.45",8.06,0.03,468,2886,6.17,"4,834.38",134.18,"2,262,490.00","2,339,215.55"
4,APC +10%,783.95,"18,548.00","149,523.45",8.06,0.03,468,2886,6.78,"5,317.82",134.18,"2,488,739.00","2,339,215.55"
5,LTC (CPA) -10%,783.95,"18,548.00","149,523.45",7.26,0.03,468,2886,6.17,"4,834.38",121.98,"2,262,490.00","2,127,918.90"


In [68]:
simulate_metric_changes_by_product(product_df, 'UX/UI Design', delta_pct=10)

,Metric ↑ / ↓ →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,813.37,"18,548.00","149,523.45",8.06,0.01,226,1170,5.18,"4,210.82",51.31,"951,645.00","802,121.55"
1,AOV +10%,894.71,"18,548.00","149,523.45",8.06,0.01,226,1170,5.18,"4,631.90",56.44,"1,046,809.50","897,286.05"
2,UA +10%,813.37,"20,402.80","149,523.45",8.06,0.01,226,1170,5.18,"4,210.82",51.31,"951,645.00","882,333.71"
3,C1 +10%,813.37,"18,548.00","149,523.45",8.06,0.01,226,1170,5.18,"4,210.82",56.44,"951,645.00","897,286.05"
4,APC +10%,813.37,"18,548.00","149,523.45",8.06,0.01,226,1170,5.69,"4,631.90",56.44,"1,046,809.50","897,286.05"
5,LTC (CPA) -10%,813.37,"18,548.00","149,523.45",7.26,0.01,226,1170,5.18,"4,210.82",51.31,"951,645.00","817,073.90"


In [69]:
simulate_metric_changes_by_product(product_df, 'Web Developer', delta_pct=10)

,Metric ↑ / ↓ →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,726.10,"18,548.00","149,523.45",8.06,0.01,135,505,3.74,"2,716.15",19.77,"366,680.00","217,156.55"
1,AOV +10%,798.71,"18,548.00","149,523.45",8.06,0.01,135,505,3.74,"2,987.76",21.75,"403,348.00","253,824.55"
2,UA +10%,726.10,"20,402.80","149,523.45",8.06,0.01,135,505,3.74,"2,716.15",19.77,"366,680.00","238,872.21"
3,C1 +10%,726.10,"18,548.00","149,523.45",8.06,0.01,135,505,3.74,"2,716.15",21.75,"366,680.00","253,824.55"
4,APC +10%,726.10,"18,548.00","149,523.45",8.06,0.01,135,505,4.11,"2,987.76",21.75,"403,348.00","253,824.55"
5,LTC (CPA) -10%,726.10,"18,548.00","149,523.45",7.26,0.01,135,505,3.74,"2,716.15",19.77,"366,680.00","232,108.90"


In [37]:
def change_metrics_table(df, product, education_type, delta_pct=10):
    increase_factor = 1 + delta_pct / 100
    decrease_factor = 1 - delta_pct / 100

    # Классификация метрик
    metrics = ['AOV', 'UA', 'AC', 'LTC (CPA)', 'C1', 'B', 'T', 'APC', 'CLTV', 'LTV', 'Revenue', 'CM']
    metrics_to_increase = ['AOV', 'UA', 'C1', 'APC']
    metrics_to_decrease = ['LTC (CPA)']
    metrics_fixed = ['AC', 'B', 'T', 'CLTV', 'LTV']  # рассчитываются, но не меняются напрямую
    row_metrics = metrics_to_increase + metrics_to_decrease

    # Получение базовых значений
    base = df[
        (df['Product'] == product) &
        (df['Education Type'] == education_type)
    ].iloc[0]
    base_values = base[metrics].to_dict()

    # Подготовка таблицы
    table = {
        'Метрика ↑ / ↓ Метрика →': ['Base'] + [
            f'{m} +{delta_pct}%' if m in metrics_to_increase else f'{m} -{delta_pct}%' for m in row_metrics
        ]
    }

    for col_metric in metrics:
        col_values = [base_values[col_metric]]

        for row_metric in row_metrics:
            temp = base_values.copy()

            # Изменение выбранной метрики
            if row_metric in metrics_to_increase:
                temp[row_metric] *= increase_factor
            elif row_metric in metrics_to_decrease:
                temp[row_metric] *= decrease_factor

            # Пересчёт зависимых метрик
            temp['CLTV'] = temp['AOV'] * temp['APC']
            temp['LTV'] = temp['CLTV'] * temp['C1']
            temp['CM'] = temp['UA'] * (temp['LTV'] - temp['LTC (CPA)'])
            temp['Revenue'] = temp['AOV'] * temp['APC'] * temp['B']

            col_values.append(temp[col_metric])

        table[col_metric] = col_values

    return pd.DataFrame(table)

In [38]:
change_metrics_table(unit_df, 'Digital Marketing', 'Evening', delta_pct=10)
# pd.set_option('display.float_format', '{:,.2f}'.format)

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,365.69,"18,548.00","149,523.45",8.06,0.01,112,764,6.82,"2,494.55",15.06,"279,390.00","129,866.55"
1,AOV +10%,402.26,"18,548.00","149,523.45",8.06,0.01,112,764,6.82,"2,744.01",16.57,"307,329.00","157,805.55"
2,UA +10%,365.69,"20,402.80","149,523.45",8.06,0.01,112,764,6.82,"2,494.55",15.06,"279,390.00","142,853.20"
3,C1 +10%,365.69,"18,548.00","149,523.45",8.06,0.01,112,764,6.82,"2,494.55",16.57,"279,390.00","157,805.55"
4,APC +10%,365.69,"18,548.00","149,523.45",8.06,0.01,112,764,7.50,"2,744.01",16.57,"307,329.00","157,805.55"
5,LTC (CPA) -10%,365.69,"18,548.00","149,523.45",7.26,0.01,112,764,6.82,"2,494.55",15.06,"279,390.00","144,818.89"


In [39]:
change_metrics_table(unit_df, 'Digital Marketing', 'Morning', delta_pct=10)

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,931.25,"18,548.00","149,523.45",8.06,0.02,349,2056,5.89,"5,486.10",103.23,"1,914,650.00","1,765,126.55"
1,AOV +10%,"1,024.38","18,548.00","149,523.45",8.06,0.02,349,2056,5.89,"6,034.71",113.55,"2,106,115.00","1,956,591.55"
2,UA +10%,931.25,"20,402.80","149,523.45",8.06,0.02,349,2056,5.89,"5,486.10",103.23,"1,914,650.00","1,941,639.21"
3,C1 +10%,931.25,"18,548.00","149,523.45",8.06,0.02,349,2056,5.89,"5,486.10",113.55,"1,914,650.00","1,956,591.55"
4,APC +10%,931.25,"18,548.00","149,523.45",8.06,0.02,349,2056,6.48,"6,034.71",113.55,"2,106,115.00","1,956,591.55"
5,LTC (CPA) -10%,931.25,"18,548.00","149,523.45",7.26,0.02,349,2056,5.89,"5,486.10",103.23,"1,914,650.00","1,780,078.89"


In [40]:
change_metrics_table(unit_df, 'UX/UI Design', 'Morning', delta_pct=10)

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,935.34,"18,548.00","149,523.45",8.06,0.01,169,886,5.24,"4,903.61",44.68,"828,710.00","679,186.55"
1,AOV +10%,"1,028.87","18,548.00","149,523.45",8.06,0.01,169,886,5.24,"5,393.97",49.15,"911,581.00","762,057.55"
2,UA +10%,935.34,"20,402.80","149,523.45",8.06,0.01,169,886,5.24,"4,903.61",44.68,"828,710.00","747,105.21"
3,C1 +10%,935.34,"18,548.00","149,523.45",8.06,0.01,169,886,5.24,"4,903.61",49.15,"828,710.00","762,057.55"
4,APC +10%,935.34,"18,548.00","149,523.45",8.06,0.01,169,886,5.77,"5,393.97",49.15,"911,581.00","762,057.55"
5,LTC (CPA) -10%,935.34,"18,548.00","149,523.45",7.26,0.01,169,886,5.24,"4,903.61",44.68,"828,710.00","694,138.90"


In [41]:
change_metrics_table(unit_df, 'UX/UI Design', 'Evening', delta_pct=10)

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,432.87,"18,548.00","149,523.45",8.06,0.00,58,284,4.90,"2,119.57",6.63,"122,935.00","-26,588.45"
1,AOV +10%,476.16,"18,548.00","149,523.45",8.06,0.00,58,284,4.90,"2,331.53",7.29,"135,228.50","-14,294.95"
2,UA +10%,432.87,"20,402.80","149,523.45",8.06,0.00,58,284,4.90,"2,119.57",6.63,"122,935.00","-29,247.30"
3,C1 +10%,432.87,"18,548.00","149,523.45",8.06,0.00,58,284,4.90,"2,119.57",7.29,"122,935.00","-14,294.95"
4,APC +10%,432.87,"18,548.00","149,523.45",8.06,0.00,58,284,5.39,"2,331.53",7.29,"135,228.50","-14,294.95"
5,LTC (CPA) -10%,432.87,"18,548.00","149,523.45",7.26,0.00,58,284,4.90,"2,119.57",6.63,"122,935.00","-11,636.11"


In [42]:
change_metrics_table(unit_df, 'UX/UI Design', 'Morning', delta_pct=10)

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,935.34,"18,548.00","149,523.45",8.06,0.01,169,886,5.24,"4,903.61",44.68,"828,710.00","679,186.55"
1,AOV +10%,"1,028.87","18,548.00","149,523.45",8.06,0.01,169,886,5.24,"5,393.97",49.15,"911,581.00","762,057.55"
2,UA +10%,935.34,"20,402.80","149,523.45",8.06,0.01,169,886,5.24,"4,903.61",44.68,"828,710.00","747,105.21"
3,C1 +10%,935.34,"18,548.00","149,523.45",8.06,0.01,169,886,5.24,"4,903.61",49.15,"828,710.00","762,057.55"
4,APC +10%,935.34,"18,548.00","149,523.45",8.06,0.01,169,886,5.77,"5,393.97",49.15,"911,581.00","762,057.55"
5,LTC (CPA) -10%,935.34,"18,548.00","149,523.45",7.26,0.01,169,886,5.24,"4,903.61",44.68,"828,710.00","694,138.90"


In [43]:
change_metrics_table(unit_df, 'Web Developer', 'Morning', delta_pct=10)	

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,726.10,"18,548.00","149,523.45",8.06,0.01,135,505,3.74,"2,716.15",19.77,"366,680.00","217,156.55"
1,AOV +10%,798.71,"18,548.00","149,523.45",8.06,0.01,135,505,3.74,"2,987.76",21.75,"403,348.00","253,824.55"
2,UA +10%,726.10,"20,402.80","149,523.45",8.06,0.01,135,505,3.74,"2,716.15",19.77,"366,680.00","238,872.21"
3,C1 +10%,726.10,"18,548.00","149,523.45",8.06,0.01,135,505,3.74,"2,716.15",21.75,"366,680.00","253,824.55"
4,APC +10%,726.10,"18,548.00","149,523.45",8.06,0.01,135,505,4.11,"2,987.76",21.75,"403,348.00","253,824.55"
5,LTC (CPA) -10%,726.10,"18,548.00","149,523.45",7.26,0.01,135,505,3.74,"2,716.15",19.77,"366,680.00","232,108.90"
